In [1]:
// 1. Installation

//install libraries
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._

// data files - generic path from docker run -v
val cdxPath = "/data/arc_cdx/*.cdx"
val warcPath = "/data/warc"

import org.apache.spark.sql.{Row, SparkSession}

val session = spark.newSession

// collect all records

val r = ArchiveSpark.load(WarcSpec.fromFiles(cdxPath, warcPath))

In [2]:
val researchYear = "2001"

In [3]:
// 2. Count ALL objects from research Year
// 2.1 get data for text/html & HTTP 200 for each Year
val t1 = r.filter(r => r.timestamp.startsWith(researchYear))


In [4]:
// 3. generate basic data frame

val m = t1.map(f=> (f.originalUrl,f.digest,f.status,f.mime))
val df = session.createDataFrame(m).toDF("originalUrl","digest","status","mime").cache()

In [5]:
import org.apache.spark.sql.functions.count
import org.apache.spark.sql.functions._


In [6]:
df.filter(df("mime") === "text/html").select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                   10705498|
+---------------------------+



In [7]:
df.select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                   13918167|
+---------------------------+



In [8]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").orderBy(desc("URLs")).show(300,false)

+--------------------------------------+--------+
|mime                                  |URLs    |
+--------------------------------------+--------+
|text/html                             |10705498|
|image/gif                             |1341197 |
|image/jpeg                            |1223622 |
|alexa/dat                             |535405  |
|text/plain                            |152274  |
|text/css                              |44280   |
|application/octet-stream              |33463   |
|image/png                             |26273   |
|application/x-javascript              |21412   |
|unk                                   |10544   |
|application/zip                       |4889    |
|text/xml                              |3427    |
|application/coreldraw                 |2092    |
|application/x-msdos-program           |2015    |
|application/pdf                       |1927    |
|audio/x-pn-realaudio                  |1635    |
|image/bmp                             |1615    |


In [9]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").count()

253